In [50]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pytz
import ta

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
from datetime import datetime
import MetaTrader5 as mt5

from ortisan_ta.dataaccess import DataItem, MetaTraderDataAccess
import ortisan_ta.utils.analysis as ortisan_ta

from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.volume import OnBalanceVolumeIndicator
from ortisan_ta.simulator import MarketSimulator


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
#Settings
mpl.style.use('seaborn')
default_fig_size = (20, 20) # Width , Height

### Symbols in my historical database

In [52]:
symbols = ["ABEV3",
"AZUL4",
"B3SA3",
"BBAS3",
"BBDC4",
"BBSE3",
"BPAC11",
"BRDT3",
"BRFS3",
"BRML3",
"BTOW3",
"CCRO3",
"CIEL3",
"CMIG4",
"COGN3",
"CSNA3",
"CVCB3",
"CYRE3",
"ELET3",
"EQTL3",
"GGBR4",
"GNDI3",
"GOAU4",
"GOLL4",
"IRBR3",
"ITSA4",
"ITUB4",
"JBSS3",
"KLBN11",
"LAME4",
"LREN3",
"MGLU3",
"MRFG3",
"MULT3",
"NTCO3",
"PETR3",
"PETR4",
"PRIO3",
"RADL3",
"RAIL3",
"RENT3",
"SBSP3",
"SULA11",
"SUZB3",
"TOTS3",
"UGPA3",
"USIM5",
"VALE3",
"VVAR3",
"WEGE3"]


In [53]:
symbols

['ABEV3',
 'AZUL4',
 'B3SA3',
 'BBAS3',
 'BBDC4',
 'BBSE3',
 'BPAC11',
 'BRDT3',
 'BRFS3',
 'BRML3',
 'BTOW3',
 'CCRO3',
 'CIEL3',
 'CMIG4',
 'COGN3',
 'CSNA3',
 'CVCB3',
 'CYRE3',
 'ELET3',
 'EQTL3',
 'GGBR4',
 'GNDI3',
 'GOAU4',
 'GOLL4',
 'IRBR3',
 'ITSA4',
 'ITUB4',
 'JBSS3',
 'KLBN11',
 'LAME4',
 'LREN3',
 'MGLU3',
 'MRFG3',
 'MULT3',
 'NTCO3',
 'PETR3',
 'PETR4',
 'PRIO3',
 'RADL3',
 'RAIL3',
 'RENT3',
 'SBSP3',
 'SULA11',
 'SUZB3',
 'TOTS3',
 'UGPA3',
 'USIM5',
 'VALE3',
 'VVAR3',
 'WEGE3']

### Gets the data for each symbol

In [63]:
data_access = MetaTraderDataAccess()
dfs = data_access.get_rates_from_symbols(symbols, datetime(2020, 1, 1), datetime(2020, 12, 4), mt5.TIMEFRAME_D1)
len(dfs)

50

### Filtering dfs with data (df is not empty)

In [64]:
dfs_with_data = dict(filter(lambda elem: not elem[1].empty, dfs.items()))

In [67]:
len(dfs_with_data)

49

### Sample

In [66]:
example_asset = "ITUB4"
df = dfs_with_data[example_asset]
df.head()

,Open,High,Low,Close,Spread,Volume
Date,,,,,,
2020-01-01 21:00:00,35.61,36.33,35.34,36.33,1,20666100
2020-01-02 21:00:00,35.82,36.53,35.78,35.95,1,24891400
2020-01-05 21:00:00,35.87,35.90,35.26,35.41,1,22294700
2020-01-06 21:00:00,35.41,35.58,34.59,34.59,1,20000900
2020-01-07 21:00:00,34.82,35.17,34.03,34.03,1,25980900


In [58]:
close = df["Close"]
open = df["Open"]
volume = df["Volume"]
high = df["High"]
low = df["Low"]

In [61]:
fig = go.Figure(data=[go.Candlestick(x=close.index, open=open, high=high, low=low, close=close)])
fig.show()

In [71]:
def get_slope_min(arr: np.ndarray):
    arg_sorted = np.argsort(arr, axis=0)
    x1 = arg_sorted[0]
    x2 = arg_sorted[1]
    y1 = arr[x1]
    y2 = arr[x2]
    slope = (y2 - y1) / (x2 - x1)
    return slope


In [81]:
def get_slope_max(arr: np.ndarray):
    arg_sorted = np.argsort(arr, axis=0)
    arg_sorted = arg_sorted[::-1]
    x1 = arg_sorted[0]
    x2 = arg_sorted[1]
    y1 = arr[x1]
    y2 = arr[x2]
    slope = (y2 - y1) / (x2 - x1)
    return slope


In [72]:
low_slopes = low.rolling(10).apply(get_slope_min, raw=True)

In [85]:
low_slopes.head(11)

Date
2020-01-01 21:00:00     NaN
2020-01-02 21:00:00     NaN
2020-01-05 21:00:00     NaN
2020-01-06 21:00:00     NaN
2020-01-07 21:00:00     NaN
2020-01-08 21:00:00     NaN
2020-01-09 21:00:00     NaN
2020-01-12 21:00:00     NaN
2020-01-13 21:00:00     NaN
2020-01-14 21:00:00   -0.03
2020-01-15 21:00:00    0.02
Name: Low, dtype: float64

In [84]:
high_slopes = high.rolling(10).apply(get_slope_max, raw=True)
high_slopes.head(11)

Date
2020-01-01 21:00:00     NaN
2020-01-02 21:00:00     NaN
2020-01-05 21:00:00     NaN
2020-01-06 21:00:00     NaN
2020-01-07 21:00:00     NaN
2020-01-08 21:00:00     NaN
2020-01-09 21:00:00     NaN
2020-01-12 21:00:00     NaN
2020-01-13 21:00:00     NaN
2020-01-14 21:00:00    0.20
2020-01-15 21:00:00   -0.63
Name: High, dtype: float64

In [88]:
divergences = low_slopes * high_slopes < 0
close_divergences = close[divergences]
close_divergences

Date
2020-01-14 21:00:00    33.08
2020-01-15 21:00:00    33.15
2020-01-16 21:00:00    33.38
2020-01-21 21:00:00    32.19
2020-01-27 21:00:00    31.99
                       ...  
2020-11-25 21:00:00    28.45
2020-11-26 21:00:00    28.67
2020-11-29 21:00:00    28.28
2020-12-01 21:00:00    29.70
2020-12-03 21:00:00    29.97
Name: Close, Length: 101, dtype: float64

In [98]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Candlestick(x=close.index, open=open, high=high, low=low, close=close), row=1, col=1)

fig.add_trace(go.Scatter(
    x=close_divergences.index, y=close_divergences, name='Closes Divergences High Low', mode='markers'), row=1, col=1)

fig.update_layout(height=1000, width=1000, title_text="Closes divergences slope", title_x=0.5)
fig.show()

In [80]:
x = np.array([1,2])
x[::-1]

array([2, 1])